In [1]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torchvision
import torch.nn as nn
import torch.nn.functional as F

In [7]:
# global datatype
dtype = torch.float32
batch_size = 64
learning_rate = 3e-4
epochs = 1
#global device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
#train, val and test data
data = torchvision.datasets.CIFAR10(root = '/home/sasuke/Desktop/deep learning', download = True, train = True, \
                                    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))
test_data = torchvision.datasets.CIFAR10(root = '/home/sasuke/Desktop/deep learning', train = False, transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))                 
train_data, val_data = torch.utils.data.random_split(data, [40000, 10000])
print(f"train:{len(train_data)}, test:{len(test_data)}, val:{len(val_data)}")

Files already downloaded and verified
train:40000, test:10000, val:10000


In [4]:
#loader
train_loader = DataLoader(train_data, batch_size = batch_size)
val_loader = DataLoader(val_data, batch_size)
test_loader = DataLoader(test_data)
#impliment num_workers, memorypinning, etc

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), padding='same')
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=(3,3), padding = 'same')
        self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size = 3, padding = 'same')
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size = 3, padding = 'same')
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(8 * 8 * 64, 128)
        self.fc2 = nn.Linear(128, 10)
    def forward(self, x):
        x = F.relu(self.conv1(x)) #32*32*48
        x = F.relu(self.conv2(x)) #32*32*96
        x = self.pool(x) #16*16*96
        x = F.relu(self.conv3(x)) #16*16*192
        x = F.relu(self.conv4(x)) #16*16*256
        x = self.pool(x) # 8*8*256
        x = self.flatten(x)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x

In [35]:
#model, optimiuzer and loss
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=0.001)
for epoch in range(epochs):
    countt = 0
    for (images, labels) in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        output = model(images)

        loss = criterion(output, labels)
        optimizer.zero_grad()
        with torch.no_grad():
            _, pred= torch.max(output, dim = 1)
            countt += (pred == labels).sum().item()
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        countv = 0
        for (image, labels) in val_loader:
            img = image.to(device)
            labels = labels.to(device)
            output = model(img)
            _, predictions = torch.max(output, dim = 1)
            countv += (predictions == labels).sum().item()



<ipython-input-5-1ffe68ae5667>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


In [39]:
with torch.no_grad():
    countv = 0
    for (image, labels) in test_loader:
        img = image.to(device)
        labels = labels.to(device)
        output = model(img)
        _, predictions = torch.max(output, dim = 1)
        countv += (predictions == labels).sum().item()
print(f"{countv/len(test_data):.3f}")

<ipython-input-5-1ffe68ae5667>:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


0.400
